In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [61]:
imp_data = pd.read_csv("../data/longimp1/import.csv", )
imp_data.head()

Country    Time  Price Index
0  Canada  19-Dec        138.0
1  Canada  20-Jan        137.1
2  Canada  20-Feb        130.8
3  Canada  20-Mar        120.5
4  Canada  20-Apr        106.3

In [62]:
imp_data['Time'] = pd.to_datetime(imp_data['Time'], format="%y-%b")

In [63]:
policy_data = pd.read_csv("../data/OxCGRT_latest.csv")
policy_data.head()

/Users/wenqianzhang/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


CountryName CountryCode RegionName RegionCode Jurisdiction      Date  \
0       Aruba         ABW        NaN        NaN    NAT_TOTAL  20200101   
1       Aruba         ABW        NaN        NaN    NAT_TOTAL  20200102   
2       Aruba         ABW        NaN        NaN    NAT_TOTAL  20200103   
3       Aruba         ABW        NaN        NaN    NAT_TOTAL  20200104   
4       Aruba         ABW        NaN        NaN    NAT_TOTAL  20200105   

   C1_School closing  C1_Flag  C2_Workplace closing  C2_Flag  ...  \
0                0.0      NaN                   0.0      NaN  ...   
1                0.0      NaN                   0.0      NaN  ...   
2                0.0      NaN                   0.0      NaN  ...   
3                0.0      NaN                   0.0      NaN  ...   
4                0.0      NaN                   0.0      NaN  ...   

   StringencyIndex  StringencyIndexForDisplay  StringencyLegacyIndex  \
0              0.0                        0.0                    0.0   
1              0.0                        0.0                    0.0   
2              0.0                        0.0                    0.0   
3              0.0                        0.0                    0.0   
4              0.0                        0.0                    0.0   

   StringencyLegacyIndexForDisplay  GovernmentResponseIndex  \
0                              0.0                      0.0   
1                              0.0                      0.0   
2                              0.0                      0.0   
3                              0.0                      0.0   
4                              0.0                      0.0   

   GovernmentResponseIndexForDisplay  ContainmentHealthIndex  \
0                                0.0                     0.0   
1                                0.0                     0.0   
2                                0.0                     0.0   
3                                0.0                     0.0   
4                                0.0                     0.0   

   ContainmentHealthIndexForDisplay  EconomicSupportIndex  \
0                               0.0                   0.0   
1                               0.0                   0.0   
2                               0.0                   0.0   
3                               0.0                   0.0   
4                               0.0                   0.0   

   EconomicSupportIndexForDisplay  
0                             0.0  
1                             0.0  
2                             0.0  
3                             0.0  
4                             0.0  

[5 rows x 61 columns]

In [64]:
policy_data.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode',
       'Jurisdiction', 'Date', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'H6_Facial Coverings', 'H6_Flag', 'H7_Vaccination policy', 'H7_Flag',
       'H8_Protection of elderly people', 'H8_Flag', 'M1_Wildcard',
       'V1_Vaccine Prioritisation (summary)',
       'V2A_Vaccine Availability (summary)',
       'V2B_Vaccin

In [65]:
cols = ['CountryName', 'RegionName', 'Date', 'C1_School closing', 'C2_Workplace closing', 'C3_Cancel public events', 
        'C4_Restrictions on gatherings', 'C5_Close public transport', 'C6_Stay at home requirements',
       'C7_Restrictions on internal movement', 'C8_International travel controls', 
        'ConfirmedCases', 'ConfirmedDeaths', 'StringencyIndex']
policy_data_new = policy_data[cols].copy()

In [66]:
countries = ['Canada', 'United States', 'Mexico', 'Japan', 'China']
policy_data_new = policy_data_new[policy_data_new['CountryName'].isin(countries)].copy()

In [67]:
policy_data_new['Date'] = pd.to_datetime(policy_data_new['Date'], format="%Y%m%d")

In [68]:
policy_data_new = policy_data_new.fillna(0)
policy_data_new['month'] = policy_data_new['Date'].apply(lambda x: str(x.year) + '-' + str(x.month))

In [69]:
agg_data = policy_data_new.groupby(['CountryName', 'month']).sum().reset_index()

In [70]:
poli = ['C1_School closing', 'C2_Workplace closing', 'C3_Cancel public events', 
        'C4_Restrictions on gatherings', 'C5_Close public transport', 'C6_Stay at home requirements',
       'C7_Restrictions on internal movement', 'C8_International travel controls']

for pol in poli:
    agg_data[pol] = agg_data[pol].apply(lambda x: 1 if x > 0 else 0)

In [71]:
agg_data['month'] = pd.to_datetime(agg_data['month'])

In [72]:
imp_data['month'] = imp_data['Time'].apply(lambda x: str(x)[: 7])
imp_data['month'] = pd.to_datetime(imp_data['month'])

In [73]:
data_all = pd.merge(imp_data, agg_data, left_on=['Country', 'month'], right_on=['CountryName', 'month'], how='left')
data_all.fillna(0, inplace=True)
data_all.drop(columns=['Time', 'CountryName'], inplace=True)

In [74]:
data_all['log_cases'] = np.log1p(data_all['ConfirmedCases'])
data_all['log_death'] = np.log1p(data_all['ConfirmedDeaths'])

In [75]:
us_policy = agg_data[agg_data['CountryName']=='United States'].copy()
us_policy['log_cases'] = np.log1p(us_policy['ConfirmedCases'])
us_policy['log_death'] = np.log1p(us_policy['ConfirmedDeaths'])

In [76]:
data_all = pd.merge(data_all, us_policy, on=['month'], how='left', suffixes=['_org', '_dst']).fillna(0)

In [77]:
data_all.to_csv('../data/processed/import_policy.csv', index=False)